In [1]:
%%javascript

IPython.tab_as_tab_everywhere = function(use_tabs) {
    if (use_tabs === undefined) {
        use_tabs = true; 
    }

    // apply setting to all current CodeMirror instances
    IPython.notebook.get_cells().map(
        function(c) {  return c.code_mirror.options.indentWithTabs=use_tabs;  }
    );
    // make sure new CodeMirror instances created in the future also use this setting
    CodeMirror.defaults.indentWithTabs=use_tabs;

    };

IPython.tab_as_tab_everywhere()

<IPython.core.display.Javascript object>

In [4]:
def final_fun_1(X):
	"""This Function will take test input and return the predicted values related to different classes"""
	import pandas as pd
	import numpy as np
	import matplotlib.pyplot as plt
	%matplotlib inline
	import seaborn as sns
	import os
	import re
	import time
	from sklearn.metrics import confusion_matrix
	from sklearn.metrics.classification import log_loss
	from sklearn.feature_extraction.text import TfidfVectorizer
	from sklearn.feature_extraction.text import CountVectorizer
	from collections import Counter
	from scipy.sparse import hstack
	from scipy.sparse import csr_matrix
	import warnings
	warnings.filterwarnings("ignore")
	from datetime import datetime
	from sklearn.model_selection import train_test_split
	from sklearn.linear_model import LogisticRegression
	from sklearn.ensemble import RandomForestClassifier
	from sklearn.calibration import CalibratedClassifierCV
	from xgboost import XGBClassifier
	from sklearn.model_selection import RandomizedSearchCV
	from sklearn.preprocessing import LabelEncoder
	from sklearn.preprocessing import StandardScaler
	from keras.models import Sequential
	from keras.layers import Dense, Dropout, Activation, BatchNormalization,Input,PReLU
	from keras.utils import np_utils
	from keras.optimizers import Adam
	from keras.models import Model
	from keras.optimizers import Adagrad
	from sklearn.externals import joblib
	start=datetime.now()
	train_data=pd.read_csv('Train_Data_with_events_details.csv')
	test_data=pd.read_csv('Test_Data_with_evetns_details.csv')
	train_data.drop(['Unnamed: 0'],axis=1,inplace=True)#removing the first column which does nt have any column name...
	test_data.drop(['Unnamed: 0'],axis=1,inplace=True)#removing the first column which does not have any column name....
	events_train_data=train_data.loc[train_data['has_events']==True]#To get data with events
	events_test_data=test_data.loc[test_data['has_events']==True]
	noevents_train_data=train_data.loc[train_data['has_events']==False]#to get data without events
	noevents_test_data=test_data.loc[test_data['has_events']==False]
	events_train_data.drop(['has_events'],axis=1,inplace=True)# we are keeping only device id by dropping has_events column
	events_test_data.drop(['has_events'],axis=1,inplace=True)
	events = pd.read_csv('events.csv',  parse_dates=['timestamp'], index_col='event_id')
	phone_data=pd.read_csv('phone_brand_device_model.csv')
	# we are droping Duplicate Devices and set Device_id as index like we did for Data while importing
	phone_data = phone_data.drop_duplicates('device_id',keep='first').set_index('device_id')
	# we take event_id, app_id and is_active.we dont take is_installed feature as it is not much useful
	app_events = pd.read_csv('app_events.csv', usecols=['event_id','app_id','is_active'], dtype={'is_active':bool})#we are treating is_active data type as boolean
	app_labels = pd.read_csv('app_labels.csv')
	label_categories = pd.read_csv('label_categories.csv')
	train_data=train_data.set_index('device_id')
	test_data=test_data.set_index('device_id')
	events_train_data=events_train_data.set_index('device_id')
	events_test_data=events_test_data.set_index('device_id')
	noevents_train_data=noevents_train_data.set_index('device_id')
	noevents_test_data=noevents_test_data.set_index('device_id')
	#we will use numpy.arange function to create row numbers 
	train_data['trainrow']=np.arange(train_data.shape[0])#all devices
	events_train_data['trainrow']=np.arange(events_train_data.shape[0])#devices with events
	test_data['testrow']=np.arange(test_data.shape[0])#all devices
	events_test_data['testrow']=np.arange(events_test_data.shape[0])#devices with events
	noevents_train_data['trainrow']=np.arange(noevents_train_data.shape[0])#devices without events
	noevents_test_data['testrow']=np.arange(noevents_test_data.shape[0])#devices without events
	#Converting Categorical Brands in phone data to Integers from 0 to number of unique brands -1
	brand_encoder = LabelEncoder().fit(phone_data['phone_brand'])
	phone_data['brand'] = brand_encoder.transform(phone_data['phone_brand'])
	nbrands=len(brand_encoder.classes_)# number of unique apps it will be used in creating One-Hot Encoding of Brands
	#Concatinating Phone Brand and Model 
	concat_model = phone_data['phone_brand'].str.cat(phone_data['device_model'])
	#Converting Categorical Maodel in phone data  to Integers from 0 to number of unique models -1
	model_encoder=LabelEncoder().fit(concat_model)
	phone_data['model_brand']=model_encoder.transform(concat_model)
	nmodels=len(model_encoder.classes_)# number of unique apps it will be used in creating One-Hot Encoding of Models
	model_encode=LabelEncoder().fit(phone_data['device_model'])
	phone_data['model']=model_encode.transform(phone_data['device_model'])
	num_models=len(model_encoder.classes_)
	#Both the dataframes have device_id as indices so phone_brand and model corresponding to a device_id get copied to appropriate
#columns
	train_data['phone_brand']=phone_data['brand']#adding encoded brand data to train data
	test_data['phone_brand']=phone_data['brand']#adding encoded brand data to test data
	train_data['phone_model']=phone_data['model']#adding encoded model data to train data
	test_data['phone_model']=phone_data['model']#adding encoded model data to test data
	events_train_data['phone_brand']=phone_data['brand']#adding encoded brand data to train data with events
	events_test_data['phone_brand']=phone_data['brand']#adding encoded brand data to test data with events
	events_train_data['phone_model']=phone_data['model_brand']#addding encoded model+brand data to train data with events
	events_test_data['phone_model']=phone_data['model_brand']#adding encoded model+brand data to test data with events
	#we dont add encoded model+brand data to nonevents data
	noevents_train_data['phone_brand']=phone_data['brand']#adding encoded brand data to train data without events
	noevents_test_data['phone_brand']=phone_data['brand']#adding encoded brand data to test data without events
	noevents_train_data['phone_model']=phone_data['model']#adding encoded model data to train data without events
	noevents_test_data['phone_model']=phone_data['model']#adding encoded model data to test data without events 
	app_encoder = LabelEncoder().fit(app_events['app_id'])
	app_events['app'] = app_encoder.transform(app_events['app_id'])
	napps = len(app_encoder.classes_)# number of unique apps it will be used in creating One-Hot Encoding of Apps 
	deviceapps = (app_events.merge(events[['device_id']], how='left',left_on='event_id',right_index=True)
                       .groupby(['device_id','app'])['app'].agg(['size'])
                       .merge(events_train_data[['trainrow']], how='left', left_index=True, right_index=True)#merges by index which is device_id from train data with events
                       .merge(events_test_data[['testrow']], how='left', left_index=True, right_index=True)#merges by index which is device_id from test data with events
                       .reset_index())
	app_labels = app_labels.loc[app_labels['app_id'].isin(app_events['app_id'].unique())]#we are taking app_id's existed both in app_events and app_labels
	app_labels['app'] = app_encoder.transform(app_labels['app_id'])#encoding common app_id's from above line
	labelencoder = LabelEncoder().fit(app_labels['label_id'])#now encoding app_labels
	app_labels['label'] = labelencoder.transform(app_labels['label_id'])
	nlabels = len(labelencoder.classes_)
	devicelabels = (deviceapps[['device_id','app']]
                .merge(app_labels[['app','label']])
                .groupby(['device_id','label'])['app'].agg(['size'])
                .merge(events_train_data[['trainrow']], how='left', left_index=True, right_index=True)#merges by index which is device_id from train data with events
                .merge(events_test_data[['testrow']], how='left', left_index=True, right_index=True)#merges by index which is device_id from test data with events
                .reset_index())#resetting index to default indices
	#Extracting Hours from Events Time Stamp by using map function 
	events['hour'] = events['timestamp'].map(lambda x:pd.to_datetime(x).hour)
	events['hourbin'] = [1 if ((x>=1)&(x<=6)) else 2 if ((x>=7)&(x<=12)) else 3 if ((x>=13)&(x<=18)) else 4 for x in events['hour']]
	hourevents = events.groupby("device_id")["hour"].apply(lambda x: " ".join('0'+str(s) for s in x))
	hourbinevents = events.groupby("device_id")["hourbin"].apply(lambda x: " ".join('0'+str(s) for s in x))
	#Mapping The Values the values to train and test dataframes
	events_train_data['event_hours']=events_train_data.index.map(hourevents)#mapping hourevents to train data with events based on index deviceid
	events_test_data['event_hours']=events_test_data.index.map(hourevents)#mapping hourevents to test data with events based on index deviceid
	events_train_data['event_hours_bins']=events_train_data.index.map(hourbinevents)#mapping houreventsbin to train data with events based on index deviceid
	events_test_data['event_hours_bins']=events_test_data.index.map(hourbinevents)#mapping houreventsbins to test data with events based on index deviceid
	#Extracting Days of the week
	days_of_week=events['timestamp'].dt.day_name()
	events['day']=days_of_week.map({'Sunday':0,'Monday':1,'Tuesday':2,'Wednesday':3,'Thursday':4,'Friday':5,'Saturday':6})#mapping day to numeric digits
	daysevents = events.groupby("device_id")["day"].apply(lambda x: " ".join('0'+str(s) for s in x))#concatening '0' to all single nubmerals
	events_train_data['event_day']=events_train_data.index.map(daysevents)#mapping days to train data with events
	events_test_data['event_day']=events_test_data.index.map(daysevents)#mapping days to test data with events
	#Grouping by device_id and taking the median of latitude
	lat_events = events.groupby("device_id")["latitude"].apply(lambda x: np.median([float(s) for s in x]))
	#Grouping by device_id and taking the median of longitude
	long_events = events.groupby("device_id")["longitude"].apply(lambda x: np.median([float(s) for s in x]))
	events_train_data['event_med_lat']=events_train_data.index.map(lat_events)#mapping lattitude_medians to events_train_data
	events_test_data['event_med_lat']=events_test_data.index.map(lat_events)#mapping lattitude_medians to events_test_data
	events_train_data['event_med_long']=events_train_data.index.map(long_events)#mapping longitude_medians to events_train_data
	events_test_data['event_med_long']=events_test_data.index.map(long_events)#mapping longitude_medians to events_test_data
	appsactive = app_events.groupby("event_id")["is_active"].apply(lambda x: " ".join(str(s) for s in x))#converting boolean to strings
	#Mapping apps is_active to device_id
	events["apps_active"] = events.index.map(appsactive)#mapping converted strings to events
	events_apps_active_map = events.groupby("device_id")["apps_active"].apply(lambda x: " ".join(str(s) for s in x if str(s)!='nan'))#mapping only non-nan(status of is_active) acitve apps
	#Mapping The Values the values to train and test dataframes
	events_train_data['apps_active']=events_train_data.index.map(events_apps_active_map)#mapping non-nan statused apps to train data with events
	events_test_data['apps_active']=events_test_data.index.map(events_apps_active_map)#mapping non-nan statused apps to test data with events
	train_data_store=events_train_data.reset_index()
	test_data_store=events_test_data.reset_index()
	train_data_noevents=noevents_train_data.reset_index()
	test_data_noevents=noevents_test_data.reset_index()
	#Storing the Processed Events Train Data
	train_data_store.to_csv('Processed_events_train_data.csv')
	#Storing the Processed Events Test
	test_data_store.to_csv('Processed_events_test_data.csv')
	#Storing the Processed No Events Train Data
	train_data_noevents.to_csv('Processed_no_events_train_data.csv')
	#Storing the Processed No Events Test
	test_data_noevents.to_csv('Processed_no_events_test_data.csv')
	#https://stackoverflow.com/questions/8955448/save-load-scipy-sparse-csr-matrix-in-portable-data-format
	def save_sparse(filename, xmtr):
		"""
		Takes the Sparse Matrix, file name as input and saves the Matrix as npz file
		"""
		np.savez(filename,data = xmtr.data ,indices= xmtr.indices,
             indptr =xmtr.indptr, shape=xmtr.shape )
	#https://stackoverflow.com/questions/8955448/save-load-scipy-sparse-csr-matrix-in-portable-data-format
	def load_sparse(filename):
		"""
		Takes the filename as input and loads the matrix stored as .npz file to a csr matrix and returns the matrix
		"""
		tmp = np.load(filename)
		return csr_matrix((tmp['data'], tmp['indices'], tmp['indptr']), shape= tmp['shape'])
	Xtr_brand = csr_matrix((np.ones(train_data.shape[0]), # Number of Rows/Devices
                       (train_data.trainrow, train_data.phone_brand)))# columns as brands 
	Xte_brand = csr_matrix((np.ones(test_data.shape[0]), # Number of Rows/Devices
                       (test_data.testrow, test_data.phone_brand)))
	Xtr_model = csr_matrix((np.ones(train_data.shape[0]), # Number of Rows/Devices
                       (train_data.trainrow, train_data.phone_model)))
	Xte_model = csr_matrix((np.ones(test_data.shape[0]), # Number of Rows/Devices
                       (test_data.testrow, test_data.phone_model)))
	X_train_one_hot=hstack((Xtr_brand,Xtr_model),format='csr')
	X_test_one_hot=hstack((Xte_brand,Xte_model),format='csr')
	save_sparse('Train_One_hot_brand_model_matrix',X_train_one_hot)
	save_sparse('Test_One_hot_brand_model_matrix',X_test_one_hot)
	#https://docs.scipy.org/doc/scipy/reference/generated/scipy.sparse.csr_matrix.html
	Xtr_noevents_brand = csr_matrix((np.ones(noevents_train_data.shape[0]), # Number of Rows/Devices
                       (noevents_train_data.trainrow, noevents_train_data.phone_brand)))
	Xte_noevents_brand = csr_matrix((np.ones(noevents_test_data.shape[0]), # Number of Rows/Devices
                       (noevents_test_data.testrow, noevents_test_data.phone_brand)))
	Xtr_noevents_model = csr_matrix((np.ones(noevents_train_data.shape[0]), # Number of Rows/Devices
                       (noevents_train_data.trainrow, noevents_train_data.phone_model)))
	Xte_noevents_model = csr_matrix((np.ones(noevents_test_data.shape[0]), # Number of Rows/Devices
                       (noevents_test_data.testrow, noevents_test_data.phone_model)))
	X_train_noevents_one_hot=hstack((Xtr_noevents_brand,Xtr_noevents_model),format='csr')
	X_test_noevents_one_hot=hstack((Xte_noevents_brand,Xte_noevents_model),format='csr')
	#Saving One-hot encoded Matrices
	save_sparse('Train_Noevents_One_hot_brand_model_matrix',X_train_noevents_one_hot)
	save_sparse('Test_Noevents_One_hot_brand_model_matrix',X_test_noevents_one_hot)
	#https://docs.scipy.org/doc/scipy/reference/generated/scipy.sparse.csr_matrix.html
	Xtr_events_brand = csr_matrix((np.ones(events_train_data.shape[0]), # Number of Rows/Devices
                       (events_train_data.trainrow, events_train_data.phone_brand)), 
                              shape=(events_train_data.shape[0],nbrands))
	Xte_events_brand = csr_matrix((np.ones(events_test_data.shape[0]), # Number of Rows/Devices
                       (events_test_data.testrow, events_test_data.phone_brand)),
                             shape=(events_test_data.shape[0],nbrands))
	#https://docs.scipy.org/doc/scipy/reference/generated/scipy.sparse.csr_matrix.html
	Xtr_events_model = csr_matrix((np.ones(events_train_data.shape[0]), # Number of Rows/Devices
                       (events_train_data.trainrow, events_train_data.phone_model)),
                         shape=(events_train_data.shape[0],nmodels))
	Xte_events_model = csr_matrix((np.ones(events_test_data.shape[0]), # Number of Rows/Devices
                       (events_test_data.testrow, events_test_data.phone_model)),
                           shape=(events_test_data.shape[0],nmodels))
	#Since the Deviceapps has both train and test columns merged to create Train Apps One-Hot we will Drop all Nan of Train Row
	#Once we remove Nan in Train Rows we will get the Apps in Train Data and we create CSR Matrix for those rows
	d = deviceapps.dropna(subset=['trainrow'])#based on trainrow we drop
	Xtr_events_app = csr_matrix((np.ones(d.shape[0]), (d.trainrow, d.app)), 
                      shape=(events_train_data.shape[0],napps))
	#Since the Deviceapps has both train and test columns merged to create Test Apps One-Hot we will Drop all Nan of Test Row
	#Once we remove Nan in Test Rows we will get the Apps in Test Data and we create CSR Matrix for those rows
	d = deviceapps.dropna(subset=['testrow'])#based on test row we drop
	Xte_events_app = csr_matrix((np.ones(d.shape[0]), (d.testrow, d.app)), 
                      shape=(events_test_data.shape[0],napps))
	#Since the Devicelabels has both train and test columns merged to create Train Labels One-Hot we will Drop all Nan of Train Row
	#Once we remove Nan in Train Rows we will get the Labels in Train Data and we create CSR Matrix for those rows
	d = devicelabels.dropna(subset=['trainrow'])##based on trainrow we drop
	Xtr_events_label = csr_matrix((np.ones(d.shape[0]), (d.trainrow, d.label)), 
                      shape=(events_train_data.shape[0],nlabels))
	#Since the Devicelabels has both train and test columns merged to create Test Labels One-Hot we will Drop all Nan of Test Row
	#Once we remove Nan in Test Rows we will get the Labels in Test Data and we create CSR Matrix for those rows
	d = devicelabels.dropna(subset=['testrow'])#based on test row  we drop
	Xte_events_label = csr_matrix((np.ones(d.shape[0]), (d.testrow, d.label)), 
                      shape=(events_test_data.shape[0],nlabels))
	vectorizer_EH=TfidfVectorizer()
	vectorizer_EH.fit(events_train_data['event_hours'].values)

	X_tr_event_hours_one_hot = vectorizer_EH.transform(events_train_data['event_hours'].values)
	X_te_event_hours_one_hot = vectorizer_EH.transform(events_test_data['event_hours'].values)
	vectorizer_EHB=CountVectorizer(binary=True)
	vectorizer_EHB.fit(events_train_data['event_hours_bins'].values)
	X_tr_event_hours_bins_one_hot = vectorizer_EHB.transform(events_train_data['event_hours_bins'].values)
	X_te_event_hours_bins_one_hot = vectorizer_EHB.transform(events_test_data['event_hours_bins'].values)
	vectorizer_ED=TfidfVectorizer()
	vectorizer_ED.fit(events_train_data['event_day'].values)
	X_tr_event_day_one_hot = vectorizer_ED.transform(events_train_data['event_day'].values)
	X_te_event_day_one_hot = vectorizer_ED.transform(events_test_data['event_day'].values)
	scaler_lat=StandardScaler()
	scaler_lat.fit(events_train_data['event_med_lat'].values.reshape(-1,1))

	X_tr_event_med_lat_scaled = scaler_lat.transform(events_train_data['event_med_lat'].values.reshape(-1,1))
	X_te_event_med_lat_scaled = scaler_lat.transform(events_test_data['event_med_lat'].values.reshape(-1,1))
	scaler_long=StandardScaler()
	scaler_long.fit(events_train_data['event_med_long'].values.reshape(-1,1))

	X_tr_event_med_long_scaled = scaler_long.transform(events_train_data['event_med_long'].values.reshape(-1,1))
	X_te_event_med_long_scaled = scaler_long.transform(events_test_data['event_med_long'].values.reshape(-1,1))
	vectorizer_AA=TfidfVectorizer()
	vectorizer_AA.fit(events_train_data['apps_active'].values)

	X_tr_apps_active_one_hot = vectorizer_AA.transform(events_train_data['apps_active'].values)
	X_te_apps_active_one_hot = vectorizer_AA.transform(events_test_data['apps_active'].values)
	#Converting to Sparse Matrices
	X_tr_event_hours_one_hot=X_tr_event_hours_one_hot.tocsr()
	X_te_event_hours_one_hot=X_te_event_hours_one_hot.tocsr()

	X_tr_event_hours_bins_one_hot=X_tr_event_hours_bins_one_hot.tocsr()
	X_te_event_hours_bins_one_hot=X_te_event_hours_bins_one_hot.tocsr()

	X_tr_event_day_one_hot=X_tr_event_day_one_hot.tocsr()
	X_te_event_day_one_hot=X_te_event_day_one_hot.tocsr()

	X_tr_apps_active_one_hot=X_tr_apps_active_one_hot.tocsr()
	X_te_apps_active_one_hot=X_te_apps_active_one_hot.tocsr()

	X_train_events_one_hot_1=hstack((Xtr_events_brand,Xtr_events_model,Xtr_events_label,X_tr_event_hours_one_hot,X_tr_event_hours_bins_one_hot,X_tr_event_day_one_hot,X_tr_event_med_lat_scaled,X_tr_event_med_long_scaled,Xtr_events_app,X_tr_apps_active_one_hot),format='csr')
	X_test_events_one_hot_1=hstack((Xte_events_brand,Xte_events_model,Xte_events_label,X_te_event_hours_one_hot,X_te_event_hours_bins_one_hot,X_te_event_day_one_hot,X_te_event_med_lat_scaled,X_te_event_med_long_scaled,Xte_events_app,X_te_apps_active_one_hot),format='csr')
	#Saving One-hot encoded Matrices
	save_sparse('Train_Events_One_hot_matrix_1',X_train_events_one_hot_1)
	save_sparse('Test_Events_One_hot_matrix_1',X_test_events_one_hot_1)
	y_data=train_data['Class'].values
	train_1, cv_1, y_train_1, y_cv_1 = train_test_split(X_train_one_hot, y_data,stratify=y_data,test_size=0.15,random_state=18)
	test_1=X_test_noevents_one_hot#because we want to test on no events data
	lr_sig_clf=joblib.load('calibrated_LR_no_events.sav')
	lr_no_events_test_prediction=lr_sig_clf.predict_proba(test_1)
	#Loading All 5 Saved Model_1_1 Neural Network  Models
	def model_1_1(input_shape):
		model = Sequential()
		model.add(Dense(256, input_dim=input_shape))
		model.add(PReLU())
		model.add(BatchNormalization())
		model.add(Dropout(0.5))
		model.add(Dense(64))
		model.add(PReLU())
		model.add(BatchNormalization())
		model.add(Dropout(0.5))
		model.add(Dense(12))
		model.add(Activation('softmax'))
		model.compile(loss='categorical_crossentropy',
              optimizer='adam',
              metrics=['accuracy'])
		return model
	def create_model_1_2(input_dim,output_dim, learRate=0.0025):
		model = Sequential()
		model.add(Dense(500, input_shape=(input_dim,), init='uniform'))
		model.add(PReLU(init='zero'))
		model.add(Dropout(0.82))
		model.add(Dense(output_dim, init='uniform'))
		model.add(Activation('softmax'))
		opt = Adagrad(lr=learRate, epsilon=1e-08)
		model.compile(loss='categorical_crossentropy',
                  optimizer=opt,
                  metrics=['accuracy'])
		return model
	def model_2_1(input_dim,output_dim):
		model = Sequential()
		model.add(Dropout(0.15, input_shape=(input_dim,)))
		model.add(Dense(240, init='uniform'))
		model.add(PReLU(init='zero'))
		model.add(Dropout(0.8))
		model.add(Dense(240, init='uniform'))
		model.add(PReLU(init='zero', weights=None))
		model.add(Dropout(0.35))
		model.add(Dense(260, init='uniform'))
		model.add(PReLU(init='zero', weights=None))
		model.add(Dropout(0.40))
		model.add(Dense(output_dim, init='uniform'))
		model.add(Activation('softmax'))
		opt = Adagrad(lr=0.008, epsilon=1e-08)
		model.compile(loss='categorical_crossentropy',
                  optimizer=opt,
                  metrics=['accuracy'])
		return model
	def model_2_2(input_dim,output_dim):
		model = Sequential()
		model.add(Dropout(0.4, input_shape=(input_dim,)))
		model.add(Dense(75))
		model.add(PReLU())
		model.add(Dropout(0.30))
		model.add(Dense(50, init='normal', activation='tanh'))
		model.add(PReLU())
		model.add(Dropout(0.20))
		model.add(Dense(output_dim, init='normal', activation='softmax'))
		model.compile(loss='categorical_crossentropy', optimizer='adadelta', metrics=['accuracy'])
		return model
	model_list_1=[]
	for i in range(5):
		model=model_1_1(train_1.shape[1])
		model.load_weights('Saved_Models/No_Events/Neural_Network_1/Model_1_1_'+str(i+1)+'.h5')
		model_list_1.append(model)
	test_pred_avg_1_1=np.zeros((test_1.shape[0],12))
	for i in range(len(model_list_1)):
		test_pred=model_list_1[i].predict_proba(test_1)
		test_pred_avg_1_1+=test_pred
	test_pred_avg_1_1/=len(model_list_1)
	#Loading Saved Model_1_2 Neural Network Model
	model_1_2=create_model_1_2(train_1.shape[1],12)
	model_1_2.load_weights('Saved_Models/No_Events/Model_1_2.h5')
	test_pred_1_2=model_1_2.predict_proba(test_1)
	y_data_events=events_train_data['Class'].values
	train_2, cv_2, y_train_2, y_cv_2 = train_test_split(X_train_events_one_hot_1, y_data_events,stratify=y_data_events,test_size=0.2,random_state=9)
	test_2=X_test_events_one_hot_1
	#Loading All 20 Saved Model_2_1 Neural Network Models
	model_list_2=[]
	for i in range(20):
		model=model_2_1(train_2.shape[1],12)
		model.load_weights('Saved_Models/Events/Neural_Network_1/Model_2_1_'+str(i+1)+'.h5')
		model_list_2.append(model)
	test_pred_avg_2_1=np.zeros((test_2.shape[0],12))
	for i in range(len(model_list_2)):
		test_pred=model_list_2[i].predict_proba(test_2)
		test_pred_avg_2_1+=test_pred
	test_pred_avg_2_1/=len(model_list_2)
	#Loading All 20 Saved Model_2_2 Neural Network Models
	model_list_3=[]
	for i in range(20):
		model=model_2_2(train_2.shape[1],12)
		model.load_weights('Saved_Models/Events/Neural_Network_2/Model_2_2_'+str(i+1)+'.h5')
		model_list_3.append(model)
	test_pred_avg_2_2=np.zeros((test_2.shape[0],12))
	for i in range(len(model_list_3)):
		test_pred=model_list_3[i].predict_proba(test_2)
		test_pred_avg_2_2+=test_pred
	test_pred_avg_2_2/=len(model_list_3)
	print("Models Predictions Done Time Taken:",datetime.now()-start)
	print("Ensembling Models......")
	w1_1=0.15
	w1_2=0.75
	w1_3=0.1
	w2_1=0.5
	w2_2=0.5
	#Esembling and Calculating weighted average predictions
	Test_Prediction_1=(w1_1*lr_no_events_test_prediction)+(w1_2*test_pred_avg_1_1)+(w1_3*test_pred_1_2)
	Test_Prediction_2=(w2_1*test_pred_avg_2_1)+(w2_2*test_pred_avg_2_2)
	print("Returned Test Predictions for Submission")
	print("Total Time Taken: ",datetime.now()-start)
	gatrain=pd.read_csv('gender_age_train.csv',index_col = 'device_id')
	targetencoder = LabelEncoder().fit(gatrain.group)
	y = targetencoder.transform(gatrain.group)
	nclasses = len(targetencoder.classes_)
	pred_1 = pd.DataFrame(Test_Prediction_1, index = noevents_test_data.index, columns=targetencoder.classes_)
	pred_2 = pd.DataFrame(Test_Prediction_2, index = events_test_data.index, columns=targetencoder.classes_)
	final_pred=pd.concat([pred_1,pred_2], axis=0)
	return final_pred


In [6]:
predicted_df=final_fun_1('gender_age_test.csv')
predicted_df.head()
predicted_df.to_csv('Submission_Final_1.csv',index=True)

Models Predictions Done Time Taken: 0:30:01.776757
Ensembling Models......
Returned Test Predictions for Submission
Total Time Taken:  0:30:01.920034


,F23-,F24-26,F27-28,F29-32,F33-42,F43+,M22-,M23-26,M27-28,M29-31,M32-38,M39+
device_id,,,,,,,,,,,,
-5893464122623104785,0.041951,0.057156,0.043716,0.067235,0.062994,0.046982,0.080628,0.153986,0.092507,0.109296,0.141558,0.101991
-7560708697029818408,0.041951,0.057156,0.043716,0.067235,0.062994,0.046982,0.080628,0.153986,0.092507,0.109296,0.141558,0.101991
289797889702373958,0.057709,0.058022,0.046506,0.068385,0.082696,0.064892,0.090617,0.115479,0.071335,0.100095,0.128950,0.115312
-402874006399730161,0.057709,0.058022,0.046506,0.068385,0.082696,0.064892,0.090617,0.115479,0.071335,0.100095,0.128950,0.115312
5751283639860028129,0.052995,0.064885,0.045246,0.068813,0.084061,0.067575,0.073340,0.124023,0.074047,0.099251,0.140322,0.105441


In [9]:
def final_fun_2(X,Y):
	"""This function will take test input and target variable then returns the multi class log-loss"""
	import pandas as pd
	import numpy as np
	import matplotlib.pyplot as plt
	%matplotlib inline
	import seaborn as sns
	import os
	import re
	import time
	from sklearn.metrics import confusion_matrix
	from sklearn.metrics.classification import log_loss
	from sklearn.feature_extraction.text import TfidfVectorizer
	from sklearn.feature_extraction.text import CountVectorizer
	from collections import Counter
	from scipy.sparse import hstack
	from scipy.sparse import csr_matrix
	import warnings
	warnings.filterwarnings("ignore")
	from datetime import datetime
	from sklearn.model_selection import train_test_split
	from sklearn.linear_model import LogisticRegression
	from sklearn.ensemble import RandomForestClassifier
	from sklearn.calibration import CalibratedClassifierCV
	from xgboost import XGBClassifier
	from sklearn.model_selection import RandomizedSearchCV
	from sklearn.preprocessing import LabelEncoder
	from sklearn.preprocessing import StandardScaler
	from keras.models import Sequential
	from keras.layers import Dense, Dropout, Activation, BatchNormalization,Input,PReLU
	from keras.utils import np_utils
	from keras.optimizers import Adam
	from keras.models import Model
	from keras.optimizers import Adagrad
	from sklearn.externals import joblib
	start=datetime.now()
	train_data=pd.read_csv('Train_Data_with_events_details.csv')
	test_data=pd.read_csv('Test_Data_with_evetns_details.csv')
	gatest1=pd.read_csv(Y)
	train_data.drop(['Unnamed: 0'],axis=1,inplace=True)#removing the first column which does nt have any column name...
	test_data.drop(['Unnamed: 0'],axis=1,inplace=True)#removing the first column which does not have any column name....
	events_train_data=train_data.loc[train_data['has_events']==True]#To get data with events
	events_test_data=test_data.loc[test_data['has_events']==True]
	noevents_train_data=train_data.loc[train_data['has_events']==False]#to get data without events
	noevents_test_data=test_data.loc[test_data['has_events']==False]
	events_train_data.drop(['has_events'],axis=1,inplace=True)# we are keeping only device id by dropping has_events column
	events_test_data.drop(['has_events'],axis=1,inplace=True)
	events = pd.read_csv('events.csv',  parse_dates=['timestamp'], index_col='event_id')
	phone_data=pd.read_csv('phone_brand_device_model.csv')
	# we are droping Duplicate Devices and set Device_id as index like we did for Data while importing
	phone_data = phone_data.drop_duplicates('device_id',keep='first').set_index('device_id')
	# we take event_id, app_id and is_active.we dont take is_installed feature as it is not much useful
	app_events = pd.read_csv('app_events.csv', usecols=['event_id','app_id','is_active'], dtype={'is_active':bool})#we are treating is_active data type as boolean
	app_labels = pd.read_csv('app_labels.csv')
	label_categories = pd.read_csv('label_categories.csv')
	train_data=train_data.set_index('device_id')
	test_data=test_data.set_index('device_id')
	events_train_data=events_train_data.set_index('device_id')
	events_test_data=events_test_data.set_index('device_id')
	noevents_train_data=noevents_train_data.set_index('device_id')
	noevents_test_data=noevents_test_data.set_index('device_id')
	#we will use numpy.arange function to create row numbers 
	train_data['trainrow']=np.arange(train_data.shape[0])#all devices
	events_train_data['trainrow']=np.arange(events_train_data.shape[0])#devices with events
	test_data['testrow']=np.arange(test_data.shape[0])#all devices
	events_test_data['testrow']=np.arange(events_test_data.shape[0])#devices with events
	noevents_train_data['trainrow']=np.arange(noevents_train_data.shape[0])#devices without events
	noevents_test_data['testrow']=np.arange(noevents_test_data.shape[0])#devices without events
	#Converting Categorical Brands in phone data to Integers from 0 to number of unique brands -1
	brand_encoder = LabelEncoder().fit(phone_data['phone_brand'])
	phone_data['brand'] = brand_encoder.transform(phone_data['phone_brand'])
	nbrands=len(brand_encoder.classes_)# number of unique apps it will be used in creating One-Hot Encoding of Brands
	#Concatinating Phone Brand and Model 
	concat_model = phone_data['phone_brand'].str.cat(phone_data['device_model'])
	#Converting Categorical Maodel in phone data  to Integers from 0 to number of unique models -1
	model_encoder=LabelEncoder().fit(concat_model)
	phone_data['model_brand']=model_encoder.transform(concat_model)
	nmodels=len(model_encoder.classes_)# number of unique apps it will be used in creating One-Hot Encoding of Models
	model_encode=LabelEncoder().fit(phone_data['device_model'])
	phone_data['model']=model_encode.transform(phone_data['device_model'])
	num_models=len(model_encoder.classes_)
	#Both the dataframes have device_id as indices so phone_brand and model corresponding to a device_id get copied to appropriate
#columns
	train_data['phone_brand']=phone_data['brand']#adding encoded brand data to train data
	test_data['phone_brand']=phone_data['brand']#adding encoded brand data to test data
	train_data['phone_model']=phone_data['model']#adding encoded model data to train data
	test_data['phone_model']=phone_data['model']#adding encoded model data to test data
	events_train_data['phone_brand']=phone_data['brand']#adding encoded brand data to train data with events
	events_test_data['phone_brand']=phone_data['brand']#adding encoded brand data to test data with events
	events_train_data['phone_model']=phone_data['model_brand']#addding encoded model+brand data to train data with events
	events_test_data['phone_model']=phone_data['model_brand']#adding encoded model+brand data to test data with events
	#we dont add encoded model+brand data to nonevents data
	noevents_train_data['phone_brand']=phone_data['brand']#adding encoded brand data to train data without events
	noevents_test_data['phone_brand']=phone_data['brand']#adding encoded brand data to test data without events
	noevents_train_data['phone_model']=phone_data['model']#adding encoded model data to train data without events
	noevents_test_data['phone_model']=phone_data['model']#adding encoded model data to test data without events 
	app_encoder = LabelEncoder().fit(app_events['app_id'])
	app_events['app'] = app_encoder.transform(app_events['app_id'])
	napps = len(app_encoder.classes_)# number of unique apps it will be used in creating One-Hot Encoding of Apps 
	deviceapps = (app_events.merge(events[['device_id']], how='left',left_on='event_id',right_index=True)
                       .groupby(['device_id','app'])['app'].agg(['size'])
                       .merge(events_train_data[['trainrow']], how='left', left_index=True, right_index=True)#merges by index which is device_id from train data with events
                       .merge(events_test_data[['testrow']], how='left', left_index=True, right_index=True)#merges by index which is device_id from test data with events
                       .reset_index())
	app_labels = app_labels.loc[app_labels['app_id'].isin(app_events['app_id'].unique())]#we are taking app_id's existed both in app_events and app_labels
	app_labels['app'] = app_encoder.transform(app_labels['app_id'])#encoding common app_id's from above line
	labelencoder = LabelEncoder().fit(app_labels['label_id'])#now encoding app_labels
	app_labels['label'] = labelencoder.transform(app_labels['label_id'])
	nlabels = len(labelencoder.classes_)
	devicelabels = (deviceapps[['device_id','app']]
                .merge(app_labels[['app','label']])
                .groupby(['device_id','label'])['app'].agg(['size'])
                .merge(events_train_data[['trainrow']], how='left', left_index=True, right_index=True)#merges by index which is device_id from train data with events
                .merge(events_test_data[['testrow']], how='left', left_index=True, right_index=True)#merges by index which is device_id from test data with events
                .reset_index())#resetting index to default indices
	#Extracting Hours from Events Time Stamp by using map function 
	events['hour'] = events['timestamp'].map(lambda x:pd.to_datetime(x).hour)
	events['hourbin'] = [1 if ((x>=1)&(x<=6)) else 2 if ((x>=7)&(x<=12)) else 3 if ((x>=13)&(x<=18)) else 4 for x in events['hour']]
	hourevents = events.groupby("device_id")["hour"].apply(lambda x: " ".join('0'+str(s) for s in x))
	hourbinevents = events.groupby("device_id")["hourbin"].apply(lambda x: " ".join('0'+str(s) for s in x))
	#Mapping The Values the values to train and test dataframes
	events_train_data['event_hours']=events_train_data.index.map(hourevents)#mapping hourevents to train data with events based on index deviceid
	events_test_data['event_hours']=events_test_data.index.map(hourevents)#mapping hourevents to test data with events based on index deviceid
	events_train_data['event_hours_bins']=events_train_data.index.map(hourbinevents)#mapping houreventsbin to train data with events based on index deviceid
	events_test_data['event_hours_bins']=events_test_data.index.map(hourbinevents)#mapping houreventsbins to test data with events based on index deviceid
	#Extracting Days of the week
	days_of_week=events['timestamp'].dt.day_name()
	events['day']=days_of_week.map({'Sunday':0,'Monday':1,'Tuesday':2,'Wednesday':3,'Thursday':4,'Friday':5,'Saturday':6})#mapping day to numeric digits
	daysevents = events.groupby("device_id")["day"].apply(lambda x: " ".join('0'+str(s) for s in x))#concatening '0' to all single nubmerals
	events_train_data['event_day']=events_train_data.index.map(daysevents)#mapping days to train data with events
	events_test_data['event_day']=events_test_data.index.map(daysevents)#mapping days to test data with events
	#Grouping by device_id and taking the median of latitude
	lat_events = events.groupby("device_id")["latitude"].apply(lambda x: np.median([float(s) for s in x]))
	#Grouping by device_id and taking the median of longitude
	long_events = events.groupby("device_id")["longitude"].apply(lambda x: np.median([float(s) for s in x]))
	events_train_data['event_med_lat']=events_train_data.index.map(lat_events)#mapping lattitude_medians to events_train_data
	events_test_data['event_med_lat']=events_test_data.index.map(lat_events)#mapping lattitude_medians to events_test_data
	events_train_data['event_med_long']=events_train_data.index.map(long_events)#mapping longitude_medians to events_train_data
	events_test_data['event_med_long']=events_test_data.index.map(long_events)#mapping longitude_medians to events_test_data
	appsactive = app_events.groupby("event_id")["is_active"].apply(lambda x: " ".join(str(s) for s in x))#converting boolean to strings
	#Mapping apps is_active to device_id
	events["apps_active"] = events.index.map(appsactive)#mapping converted strings to events
	events_apps_active_map = events.groupby("device_id")["apps_active"].apply(lambda x: " ".join(str(s) for s in x if str(s)!='nan'))#mapping only non-nan(status of is_active) acitve apps
	#Mapping The Values the values to train and test dataframes
	events_train_data['apps_active']=events_train_data.index.map(events_apps_active_map)#mapping non-nan statused apps to train data with events
	events_test_data['apps_active']=events_test_data.index.map(events_apps_active_map)#mapping non-nan statused apps to test data with events
	train_data_store=events_train_data.reset_index()
	test_data_store=events_test_data.reset_index()
	train_data_noevents=noevents_train_data.reset_index()
	test_data_noevents=noevents_test_data.reset_index()
	#Storing the Processed Events Train Data
	train_data_store.to_csv('Processed_events_train_data.csv')
	#Storing the Processed Events Test
	test_data_store.to_csv('Processed_events_test_data.csv')
	#Storing the Processed No Events Train Data
	train_data_noevents.to_csv('Processed_no_events_train_data.csv')
	#Storing the Processed No Events Test
	test_data_noevents.to_csv('Processed_no_events_test_data.csv')
	#https://stackoverflow.com/questions/8955448/save-load-scipy-sparse-csr-matrix-in-portable-data-format
	def save_sparse(filename, xmtr):
		"""
		Takes the Sparse Matrix, file name as input and saves the Matrix as npz file
		"""
		np.savez(filename,data = xmtr.data ,indices= xmtr.indices,
             indptr =xmtr.indptr, shape=xmtr.shape )
	#https://stackoverflow.com/questions/8955448/save-load-scipy-sparse-csr-matrix-in-portable-data-format
	def load_sparse(filename):
		"""
		Takes the filename as input and loads the matrix stored as .npz file to a csr matrix and returns the matrix
		"""
		tmp = np.load(filename)
		return csr_matrix((tmp['data'], tmp['indices'], tmp['indptr']), shape= tmp['shape'])
	Xtr_brand = csr_matrix((np.ones(train_data.shape[0]), # Number of Rows/Devices
                       (train_data.trainrow, train_data.phone_brand)))# columns as brands 
	Xte_brand = csr_matrix((np.ones(test_data.shape[0]), # Number of Rows/Devices
                       (test_data.testrow, test_data.phone_brand)))
	Xtr_model = csr_matrix((np.ones(train_data.shape[0]), # Number of Rows/Devices
                       (train_data.trainrow, train_data.phone_model)))
	Xte_model = csr_matrix((np.ones(test_data.shape[0]), # Number of Rows/Devices
                       (test_data.testrow, test_data.phone_model)))
	X_train_one_hot=hstack((Xtr_brand,Xtr_model),format='csr')
	X_test_one_hot=hstack((Xte_brand,Xte_model),format='csr')
	save_sparse('Train_One_hot_brand_model_matrix',X_train_one_hot)
	save_sparse('Test_One_hot_brand_model_matrix',X_test_one_hot)
	#https://docs.scipy.org/doc/scipy/reference/generated/scipy.sparse.csr_matrix.html
	Xtr_noevents_brand = csr_matrix((np.ones(noevents_train_data.shape[0]), # Number of Rows/Devices
                       (noevents_train_data.trainrow, noevents_train_data.phone_brand)))
	Xte_noevents_brand = csr_matrix((np.ones(noevents_test_data.shape[0]), # Number of Rows/Devices
                       (noevents_test_data.testrow, noevents_test_data.phone_brand)))
	Xtr_noevents_model = csr_matrix((np.ones(noevents_train_data.shape[0]), # Number of Rows/Devices
                       (noevents_train_data.trainrow, noevents_train_data.phone_model)))
	Xte_noevents_model = csr_matrix((np.ones(noevents_test_data.shape[0]), # Number of Rows/Devices
                       (noevents_test_data.testrow, noevents_test_data.phone_model)))
	X_train_noevents_one_hot=hstack((Xtr_noevents_brand,Xtr_noevents_model),format='csr')
	X_test_noevents_one_hot=hstack((Xte_noevents_brand,Xte_noevents_model),format='csr')
	#Saving One-hot encoded Matrices
	save_sparse('Train_Noevents_One_hot_brand_model_matrix',X_train_noevents_one_hot)
	save_sparse('Test_Noevents_One_hot_brand_model_matrix',X_test_noevents_one_hot)
	#https://docs.scipy.org/doc/scipy/reference/generated/scipy.sparse.csr_matrix.html
	Xtr_events_brand = csr_matrix((np.ones(events_train_data.shape[0]), # Number of Rows/Devices
                       (events_train_data.trainrow, events_train_data.phone_brand)), 
                              shape=(events_train_data.shape[0],nbrands))
	Xte_events_brand = csr_matrix((np.ones(events_test_data.shape[0]), # Number of Rows/Devices
                       (events_test_data.testrow, events_test_data.phone_brand)),
                             shape=(events_test_data.shape[0],nbrands))
	#https://docs.scipy.org/doc/scipy/reference/generated/scipy.sparse.csr_matrix.html
	Xtr_events_model = csr_matrix((np.ones(events_train_data.shape[0]), # Number of Rows/Devices
                       (events_train_data.trainrow, events_train_data.phone_model)),
                         shape=(events_train_data.shape[0],nmodels))
	Xte_events_model = csr_matrix((np.ones(events_test_data.shape[0]), # Number of Rows/Devices
                       (events_test_data.testrow, events_test_data.phone_model)),
                           shape=(events_test_data.shape[0],nmodels))
	#Since the Deviceapps has both train and test columns merged to create Train Apps One-Hot we will Drop all Nan of Train Row
	#Once we remove Nan in Train Rows we will get the Apps in Train Data and we create CSR Matrix for those rows
	d = deviceapps.dropna(subset=['trainrow'])#based on trainrow we drop
	Xtr_events_app = csr_matrix((np.ones(d.shape[0]), (d.trainrow, d.app)), 
                      shape=(events_train_data.shape[0],napps))
	#Since the Deviceapps has both train and test columns merged to create Test Apps One-Hot we will Drop all Nan of Test Row
	#Once we remove Nan in Test Rows we will get the Apps in Test Data and we create CSR Matrix for those rows
	d = deviceapps.dropna(subset=['testrow'])#based on test row we drop
	Xte_events_app = csr_matrix((np.ones(d.shape[0]), (d.testrow, d.app)), 
                      shape=(events_test_data.shape[0],napps))
	#Since the Devicelabels has both train and test columns merged to create Train Labels One-Hot we will Drop all Nan of Train Row
	#Once we remove Nan in Train Rows we will get the Labels in Train Data and we create CSR Matrix for those rows
	d = devicelabels.dropna(subset=['trainrow'])##based on trainrow we drop
	Xtr_events_label = csr_matrix((np.ones(d.shape[0]), (d.trainrow, d.label)), 
                      shape=(events_train_data.shape[0],nlabels))
	#Since the Devicelabels has both train and test columns merged to create Test Labels One-Hot we will Drop all Nan of Test Row
	#Once we remove Nan in Test Rows we will get the Labels in Test Data and we create CSR Matrix for those rows
	d = devicelabels.dropna(subset=['testrow'])#based on test row  we drop
	Xte_events_label = csr_matrix((np.ones(d.shape[0]), (d.testrow, d.label)), 
                      shape=(events_test_data.shape[0],nlabels))
	vectorizer_EH=TfidfVectorizer()
	vectorizer_EH.fit(events_train_data['event_hours'].values)

	X_tr_event_hours_one_hot = vectorizer_EH.transform(events_train_data['event_hours'].values)
	X_te_event_hours_one_hot = vectorizer_EH.transform(events_test_data['event_hours'].values)
	vectorizer_EHB=CountVectorizer(binary=True)
	vectorizer_EHB.fit(events_train_data['event_hours_bins'].values)
	X_tr_event_hours_bins_one_hot = vectorizer_EHB.transform(events_train_data['event_hours_bins'].values)
	X_te_event_hours_bins_one_hot = vectorizer_EHB.transform(events_test_data['event_hours_bins'].values)
	vectorizer_ED=TfidfVectorizer()
	vectorizer_ED.fit(events_train_data['event_day'].values)
	X_tr_event_day_one_hot = vectorizer_ED.transform(events_train_data['event_day'].values)
	X_te_event_day_one_hot = vectorizer_ED.transform(events_test_data['event_day'].values)
	scaler_lat=StandardScaler()
	scaler_lat.fit(events_train_data['event_med_lat'].values.reshape(-1,1))

	X_tr_event_med_lat_scaled = scaler_lat.transform(events_train_data['event_med_lat'].values.reshape(-1,1))
	X_te_event_med_lat_scaled = scaler_lat.transform(events_test_data['event_med_lat'].values.reshape(-1,1))
	scaler_long=StandardScaler()
	scaler_long.fit(events_train_data['event_med_long'].values.reshape(-1,1))

	X_tr_event_med_long_scaled = scaler_long.transform(events_train_data['event_med_long'].values.reshape(-1,1))
	X_te_event_med_long_scaled = scaler_long.transform(events_test_data['event_med_long'].values.reshape(-1,1))
	vectorizer_AA=TfidfVectorizer()
	vectorizer_AA.fit(events_train_data['apps_active'].values)

	X_tr_apps_active_one_hot = vectorizer_AA.transform(events_train_data['apps_active'].values)
	X_te_apps_active_one_hot = vectorizer_AA.transform(events_test_data['apps_active'].values)
	#Converting to Sparse Matrices
	X_tr_event_hours_one_hot=X_tr_event_hours_one_hot.tocsr()
	X_te_event_hours_one_hot=X_te_event_hours_one_hot.tocsr()

	X_tr_event_hours_bins_one_hot=X_tr_event_hours_bins_one_hot.tocsr()
	X_te_event_hours_bins_one_hot=X_te_event_hours_bins_one_hot.tocsr()

	X_tr_event_day_one_hot=X_tr_event_day_one_hot.tocsr()
	X_te_event_day_one_hot=X_te_event_day_one_hot.tocsr()

	X_tr_apps_active_one_hot=X_tr_apps_active_one_hot.tocsr()
	X_te_apps_active_one_hot=X_te_apps_active_one_hot.tocsr()

	X_train_events_one_hot_1=hstack((Xtr_events_brand,Xtr_events_model,Xtr_events_label,X_tr_event_hours_one_hot,X_tr_event_hours_bins_one_hot,X_tr_event_day_one_hot,X_tr_event_med_lat_scaled,X_tr_event_med_long_scaled,Xtr_events_app,X_tr_apps_active_one_hot),format='csr')
	X_test_events_one_hot_1=hstack((Xte_events_brand,Xte_events_model,Xte_events_label,X_te_event_hours_one_hot,X_te_event_hours_bins_one_hot,X_te_event_day_one_hot,X_te_event_med_lat_scaled,X_te_event_med_long_scaled,Xte_events_app,X_te_apps_active_one_hot),format='csr')
	#Saving One-hot encoded Matrices
	save_sparse('Train_Events_One_hot_matrix_1',X_train_events_one_hot_1)
	save_sparse('Test_Events_One_hot_matrix_1',X_test_events_one_hot_1)
	y_data=train_data['Class'].values
	train_1, cv_1, y_train_1, y_cv_1 = train_test_split(X_train_one_hot, y_data,stratify=y_data,test_size=0.15,random_state=18)
	test_1=X_test_noevents_one_hot#because we want to test on no events data
	lr_sig_clf=joblib.load('calibrated_LR_no_events.sav')
	lr_no_events_test_prediction=lr_sig_clf.predict_proba(test_1)
	#Loading All 5 Saved Model_1_1 Neural Network  Models
	def model_1_1(input_shape):
		model = Sequential()
		model.add(Dense(256, input_dim=input_shape))
		model.add(PReLU())
		model.add(BatchNormalization())
		model.add(Dropout(0.5))
		model.add(Dense(64))
		model.add(PReLU())
		model.add(BatchNormalization())
		model.add(Dropout(0.5))
		model.add(Dense(12))
		model.add(Activation('softmax'))
		model.compile(loss='categorical_crossentropy',
              optimizer='adam',
              metrics=['accuracy'])
		return model
	def create_model_1_2(input_dim,output_dim, learRate=0.0025):
		model = Sequential()
		model.add(Dense(500, input_shape=(input_dim,), init='uniform'))
		model.add(PReLU(init='zero'))
		model.add(Dropout(0.82))
		model.add(Dense(output_dim, init='uniform'))
		model.add(Activation('softmax'))
		opt = Adagrad(lr=learRate, epsilon=1e-08)
		model.compile(loss='categorical_crossentropy',
                  optimizer=opt,
                  metrics=['accuracy'])
		return model
	def model_2_1(input_dim,output_dim):
		model = Sequential()
		model.add(Dropout(0.15, input_shape=(input_dim,)))
		model.add(Dense(240, init='uniform'))
		model.add(PReLU(init='zero'))
		model.add(Dropout(0.8))
		model.add(Dense(240, init='uniform'))
		model.add(PReLU(init='zero', weights=None))
		model.add(Dropout(0.35))
		model.add(Dense(260, init='uniform'))
		model.add(PReLU(init='zero', weights=None))
		model.add(Dropout(0.40))
		model.add(Dense(output_dim, init='uniform'))
		model.add(Activation('softmax'))
		opt = Adagrad(lr=0.008, epsilon=1e-08)
		model.compile(loss='categorical_crossentropy',
                  optimizer=opt,
                  metrics=['accuracy'])
		return model
	def model_2_2(input_dim,output_dim):
		model = Sequential()
		model.add(Dropout(0.4, input_shape=(input_dim,)))
		model.add(Dense(75))
		model.add(PReLU())
		model.add(Dropout(0.30))
		model.add(Dense(50, init='normal', activation='tanh'))
		model.add(PReLU())
		model.add(Dropout(0.20))
		model.add(Dense(output_dim, init='normal', activation='softmax'))
		model.compile(loss='categorical_crossentropy', optimizer='adadelta', metrics=['accuracy'])
		return model
	model_list_1=[]
	for i in range(5):
		model=model_1_1(train_1.shape[1])
		model.load_weights('Saved_Models/No_Events/Neural_Network_1/Model_1_1_'+str(i+1)+'.h5')
		model_list_1.append(model)
	test_pred_avg_1_1=np.zeros((test_1.shape[0],12))
	for i in range(len(model_list_1)):
		test_pred=model_list_1[i].predict_proba(test_1)
		test_pred_avg_1_1+=test_pred
	test_pred_avg_1_1/=len(model_list_1)
	#Loading Saved Model_1_2 Neural Network Model
	model_1_2=create_model_1_2(train_1.shape[1],12)
	model_1_2.load_weights('Saved_Models/No_Events/Model_1_2.h5')
	test_pred_1_2=model_1_2.predict_proba(test_1)
	y_data_events=events_train_data['Class'].values
	train_2, cv_2, y_train_2, y_cv_2 = train_test_split(X_train_events_one_hot_1, y_data_events,stratify=y_data_events,test_size=0.2,random_state=9)
	test_2=X_test_events_one_hot_1
	#Loading All 20 Saved Model_2_1 Neural Network Models
	model_list_2=[]
	for i in range(20):
		model=model_2_1(train_2.shape[1],12)
		model.load_weights('Saved_Models/Events/Neural_Network_1/Model_2_1_'+str(i+1)+'.h5')
		model_list_2.append(model)
	test_pred_avg_2_1=np.zeros((test_2.shape[0],12))
	for i in range(len(model_list_2)):
		test_pred=model_list_2[i].predict_proba(test_2)
		test_pred_avg_2_1+=test_pred
	test_pred_avg_2_1/=len(model_list_2)
	#Loading All 20 Saved Model_2_2 Neural Network Models
	model_list_3=[]
	for i in range(20):
		model=model_2_2(train_2.shape[1],12)
		model.load_weights('Saved_Models/Events/Neural_Network_2/Model_2_2_'+str(i+1)+'.h5')
		model_list_3.append(model)
	test_pred_avg_2_2=np.zeros((test_2.shape[0],12))
	for i in range(len(model_list_3)):
		test_pred=model_list_3[i].predict_proba(test_2)
		test_pred_avg_2_2+=test_pred
	test_pred_avg_2_2/=len(model_list_3)
	print("Models Predictions Done Time Taken:",datetime.now()-start)
	print("Ensembling Models......")
	w1_1=0.15
	w1_2=0.75
	w1_3=0.1
	w2_1=0.5
	w2_2=0.5
	#Esembling and Calculating weighted average predictions
	Test_Prediction_1=(w1_1*lr_no_events_test_prediction)+(w1_2*test_pred_avg_1_1)+(w1_3*test_pred_1_2)
	Test_Prediction_2=(w2_1*test_pred_avg_2_1)+(w2_2*test_pred_avg_2_2)
	print("Returned Test Predictions for Submission")
	print("Total Time Taken: ",datetime.now()-start)
	gatrain=pd.read_csv('gender_age_train.csv',index_col = 'device_id')
	targetencoder = LabelEncoder().fit(gatrain.group)
	y = targetencoder.transform(gatrain.group)
	nclasses = len(targetencoder.classes_)
	pred_1 = pd.DataFrame(Test_Prediction_1, index = noevents_test_data.index, columns=targetencoder.classes_)
	pred_2 = pd.DataFrame(Test_Prediction_2, index = events_test_data.index, columns=targetencoder.classes_)
	final_pred=pd.concat([pred_1,pred_2], axis=0)
	targetencoder1 = LabelEncoder().fit(gatest1.group)
	y_actual= targetencoder1.transform(gatest1.group)
	return log_loss(y_actual,final_pred)



In [10]:
final_fun_2('gender_age_test.csv','test_age_group.csv')

Models Predictions Done Time Taken: 0:27:02.235947
Ensembling Models......
Returned Test Predictions for Submission
Total Time Taken:  0:27:02.360937


2.6680995509727947